In [1]:
!pip install numpy pandas scikit-surprise sklearn seaborn matplotlib datetime


     |████████████████████████████████| 11.8MB 324kB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 256kB 42.2MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617634 sha256=e9404aaff6f6a4b10a57495c3173401d14c9e1e8ee3eb8a86779c38b65de3ff5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from IPython.display import Image
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from surprise import SVD, Dataset, Reader
from surprise.model_selection import PredefinedKFold
from collections import defaultdict
from surprise.accuracy import rmse
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline 
sns.set(style="ticks")

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [71]:
df = pd.read_csv('/content/gdrive/My Drive/MMO/anime.csv')
#df1=df.drop(columns=['English name','Japanese name','Type', 'Aired', 'Premiered',	'Producers',	'Licensors'])
df=df.drop(columns=['MAL_ID','English name','Japanese name','Type', 'Aired', 'Premiered',	'Producers',	'Licensors'])
#df.drop_duplicates(subset=['Name'])
df.head (10)

,Name,Score,Genders,Episodes,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",1,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26,Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",26,Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",52,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
5,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",145,Gallop,Manga,23 min. per ep.,PG-13 - Teens 13 or older,604.0,1003,148259,2066,13907,78349,14228,11573,30202,9226.0,14904.0,22811.0,16734.0,6206.0,2621.0,795.0,336.0,140.0,151.0
6,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life",24,J.C.Staff,Manga,23 min. per ep.,PG-13 - Teens 13 or older,468.0,687,214499,4101,11909,81145,11901,11026,98518,11829.0,16309.0,20008.0,13062.0,5574.0,3148.0,1339.0,484.0,278.0,321.0
7,Hungry Heart: Wild Striker,7.59,"Slice of Life, Comedy, Sports, Shounen",52,Nippon Animation,Manga,23 min. per ep.,PG-13 - Teens 13 or older,1317.0,3612,20470,231,817,13778,828,1168,3879,1123.0,1777.0,3102.0,3075.0,1286.0,602.0,218.0,88.0,31.0,32.0
8,Initial D Fourth Stage,8.15,"Action, Cars, Sports, Drama, Seinen",24,A.C.G.T.,Manga,27 min. per ep.,PG-13 - Teens 13 or older,360.0,1233,117929,979,6082,90967,3053,1356,16471,10948.0,15820.0,22379.0,12912.0,3874.0,1236.0,369.0,97.0,48.0,259.0
9,Monster,8.76,"Drama, Horror, Mystery, Police, Psychological,...",74,Madhouse,Manga,24 min. per ep.,R+ - Mild Nudity,30.0,169,614100,29436,64648,214491,47488,23008,264465,77350.0,60652.0,43459.0,22045.0,8861.0,4381.0,2086.0,882.0,593.0,1177.0


In [72]:
df.shape

(17562, 27)

In [65]:
name = df['Name'].values
name[0:30]

array(['Cowboy Bebop', 'Cowboy Bebop: Tengoku no Tobira', 'Trigun',
       'Witch Hunter Robin', 'Bouken Ou Beet', 'Eyeshield 21',
       'Hachimitsu to Clover', 'Hungry Heart: Wild Striker',
       'Initial D Fourth Stage', 'Monster', 'Naruto', 'One Piece',
       'Tennis no Ouji-sama', 'Ring ni Kakero 1', 'School Rumble',
       'Sunabouzu', 'Texhnolyze', 'Trinity Blood', 'Yakitate!! Japan',
       'Zipang', 'Neon Genesis Evangelion',
       'Neon Genesis Evangelion: Death & Rebirth',
       'Neon Genesis Evangelion: The End of Evangelion',
       'Kenpuu Denki Berserk', 'Koukaku Kidoutai',
       'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
       'Rurouni Kenshin: Meiji Kenkaku Romantan',
       'Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka',
       'Akira', '.hack//Sign'], dtype=object)

In [7]:
gender=df['Genders'].values
gender[0:30]

array(['Action, Adventure, Comedy, Drama, Sci-Fi, Space',
       'Action, Drama, Mystery, Sci-Fi, Space',
       'Action, Sci-Fi, Adventure, Comedy, Drama, Shounen',
       'Action, Mystery, Police, Supernatural, Drama, Magic',
       'Adventure, Fantasy, Shounen, Supernatural',
       'Action, Sports, Comedy, Shounen',
       'Comedy, Drama, Josei, Romance, Slice of Life',
       'Slice of Life, Comedy, Sports, Shounen',
       'Action, Cars, Sports, Drama, Seinen',
       'Drama, Horror, Mystery, Police, Psychological, Seinen, Thriller',
       'Action, Adventure, Comedy, Super Power, Martial Arts, Shounen',
       'Action, Adventure, Comedy, Super Power, Drama, Fantasy, Shounen',
       'Action, Comedy, Sports, School, Shounen',
       'Action, Shounen, Sports', 'Comedy, Romance, School, Shounen',
       'Action, Adventure, Comedy, Ecchi, Sci-Fi, Shounen',
       'Action, Sci-Fi, Psychological, Drama',
       'Action, Supernatural, Vampire', 'Comedy, Shounen',
       'Action, Milita

In [8]:
rating=df['Rating'].values
rating[0:30]

array(['R - 17+ (violence & profanity)', 'R - 17+ (violence & profanity)',
       'PG-13 - Teens 13 or older', 'PG-13 - Teens 13 or older',
       'PG - Children', 'PG-13 - Teens 13 or older',
       'PG-13 - Teens 13 or older', 'PG-13 - Teens 13 or older',
       'PG-13 - Teens 13 or older', 'R+ - Mild Nudity',
       'PG-13 - Teens 13 or older', 'PG-13 - Teens 13 or older',
       'PG-13 - Teens 13 or older', 'PG - Children',
       'PG-13 - Teens 13 or older', 'R - 17+ (violence & profanity)',
       'R+ - Mild Nudity', 'R - 17+ (violence & profanity)',
       'PG-13 - Teens 13 or older', 'PG-13 - Teens 13 or older',
       'PG-13 - Teens 13 or older', 'R - 17+ (violence & profanity)',
       'R+ - Mild Nudity', 'R+ - Mild Nudity', 'R+ - Mild Nudity',
       'R - 17+ (violence & profanity)', 'PG-13 - Teens 13 or older',
       'R - 17+ (violence & profanity)', 'R+ - Mild Nudity',
       'PG-13 - Teens 13 or older'], dtype=object)

In [10]:
%%time
tfidfv = TfidfVectorizer()
genders_matrix = tfidfv.fit_transform(gender)
genders_matrix

CPU times: user 97.4 ms, sys: 0 ns, total: 97.4 ms
Wall time: 109 ms


# **Фильтрация на основе содержания по жанрам**

In [11]:
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_Name, X_Genders, X_Rating):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_Name - массив названий объектов
        X_Genders - массив жанров объектов
        X_Rating - массив возрастного ограничения объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'Name': pd.Series(X_Name, dtype='str'),
            'Gender': pd.Series(X_Genders, dtype='str'),
            'Rating': pd.Series(X_Rating, dtype='str'),
            'dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

In [12]:
trinity_blood_ind =17
name[trinity_blood_ind]

'Trinity Blood'

In [13]:
trinity_blood_matrix = genders_matrix[trinity_blood_ind]
trinity_blood_matrix

<1x48 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [14]:
skr1 = SimpleKNNRecommender(genders_matrix, name, gender, rating)

Выведем 10 аниме похожих на "кровь триединства"

In [67]:
df_new = df[['Name','Genders', 'Rating']]
df_new.loc[df_new['Name']=='Trinity Blood']

,Name,Genders,Rating
17,Trinity Blood,"Action, Supernatural, Vampire",R - 17+ (violence & profanity)


In [68]:
# в порядке убывания схожести на основе косинусного сходства
rec1 = skr1.recommend_for_single_object(10, trinity_blood_matrix)
rec1

,Name,Gender,Rating,dist
15595,Yichang Shengwu Jianwenlu,"Supernatural, Vampire",PG-13 - Teens 13 or older,938347.176320
16615,Noblesse,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),906943.306038
11461,Noblesse: Awakening,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),906943.306038
1644,Master Mosquiton '99,"Action, Adventure, Comedy, Supernatural, Vampire",PG-13 - Teens 13 or older,902269.296938
6104,Nyanpire The Animation,"Comedy, Supernatural, Vampire",G - All Ages,897786.169580
14352,Sirius,"Action, Historical, Supernatural, Vampire",R - 17+ (violence & profanity),889297.900037
5133,Hellsing: Psalm of the Darkness,"Action, Supernatural, Vampire, Seinen",R - 17+ (violence & profanity),873084.219101
7168,JoJo no Kimyou na Bouken (TV),"Action, Adventure, Supernatural, Vampire, Shounen",R - 17+ (violence & profanity),866381.795488
11103,Kizumonogatari III: Reiketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),866273.999806
11102,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),866273.999806


In [69]:
# При поиске с помощью Евклидова расстояния получаем почти такой же результат. Разница во 2 и 3 строке, но расстояния у них одинаковые
rec2 = skr1.recommend_for_single_object(10, trinity_blood_matrix, cos_flag = False)
rec2

,Name,Gender,Rating,dist
15595,Yichang Shengwu Jianwenlu,"Supernatural, Vampire",PG-13 - Teens 13 or older,351149.038671
11461,Noblesse: Awakening,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),431408.609005
16615,Noblesse,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),431408.609005
1644,Master Mosquiton '99,"Action, Adventure, Comedy, Supernatural, Vampire",PG-13 - Teens 13 or older,442110.174192
6104,Nyanpire The Animation,"Comedy, Supernatural, Vampire",G - All Ages,452136.772271
14352,Sirius,"Action, Historical, Supernatural, Vampire",R - 17+ (violence & profanity),470536.077177
5133,Hellsing: Psalm of the Darkness,"Action, Supernatural, Vampire, Seinen",R - 17+ (violence & profanity),503816.992368
7168,JoJo no Kimyou na Bouken (TV),"Action, Adventure, Supernatural, Vampire, Shounen",R - 17+ (violence & profanity),516949.135819
11103,Kizumonogatari III: Reiketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),517157.616582
11102,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),517157.616582


In [70]:
# Манхэттэнское расстояние дает приблизительно равные результаты поиска
rec3 = skr1.recommend_for_single_object(10, trinity_blood_matrix, 
                                        cos_flag = False, manh_flag = True)
rec3

,Name,Gender,Rating,dist
15595,Yichang Shengwu Jianwenlu,"Supernatural, Vampire",PG-13 - Teens 13 or older,430178.058232
11461,Noblesse: Awakening,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),573076.976390
16615,Noblesse,"Action, Supernatural, Vampire, School",R - 17+ (violence & profanity),573076.976390
14352,Sirius,"Action, Historical, Supernatural, Vampire",R - 17+ (violence & profanity),637941.562929
6104,Nyanpire The Animation,"Comedy, Supernatural, Vampire",G - All Ages,661777.343822
5133,Hellsing: Psalm of the Darkness,"Action, Supernatural, Vampire, Seinen",R - 17+ (violence & profanity),694635.629757
11103,Kizumonogatari III: Reiketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),717746.413634
11102,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",R - 17+ (violence & profanity),717746.413634
1644,Master Mosquiton '99,"Action, Adventure, Comedy, Supernatural, Vampire",PG-13 - Teens 13 or older,762750.970781
11524,Answer,"Music, Supernatural, Vampire",G - All Ages,791202.630915


# **Коллаборативная фильтрация. Метод user-based**

In [19]:
df_user = pd.read_csv('/content/gdrive/My Drive/MMO/ratings.csv')
df_user.head (30)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [20]:
# Количество уникальных пользователей
len(df_user['userId'].unique())

1726

In [21]:
# Количество уникальных аниме
len(df_user['movieId'].unique())

10475

In [22]:
# Сформируем матрицу взаимодействий на основе рейтингов
def create_utility_matrix(data):
    itemField = 'movieId'
    userField = 'userId'
    valueField = 'rating'  
    
    userList = data[userField].tolist()
    itemList = data[itemField].tolist()
    valueList = data[valueField].tolist()    
    
    users = list(set(userList))
    items = list(set(itemList))    
    
    users_index = {users[i]: i for i in range(len(users))}    
    pd_dict = {item: [0.0 for i in range(len(users))] for item in items}    
    
    for i in range(0,data.shape[0]):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]    
        pd_dict[item][users_index[user]] = value    
    
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    
    return X, users_index, items_index

In [23]:
%%time
user_item_matrix, users_index, items_index = create_utility_matrix(df_user)

CPU times: user 4.7 s, sys: 645 ms, total: 5.35 s
Wall time: 4.92 s


In [24]:
user_item_matrix

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32792,31,32,30,34,35,36,65567,38,39,40,...,65352,98120,98122,32587,98124,32589,32584,32591,65359,163663,32596,32598,32600,32602,98154,130922,98175,32649,65418,130956,98193,32657,32659,98203,130978,32686,65465,98239,98243,131013,131015,98248,32721,163809,98275,98279,65514,131050,65518,98296
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1724,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Выделение тестовой строки
user_item_matrix__test = user_item_matrix.loc[[1726]]
user_item_matrix__test

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32792,31,32,30,34,35,36,65567,38,39,40,...,65352,98120,98122,32587,98124,32589,32584,32591,65359,163663,32596,32598,32600,32602,98154,130922,98175,32649,65418,130956,98193,32657,32659,98203,130978,32686,65465,98239,98243,131013,131015,98248,32721,163809,98275,98279,65514,131050,65518,98296
1726,1.5,2.5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Оставшаяся часть матрицы для обучения
user_item_matrix__train = user_item_matrix.loc[:1725]
user_item_matrix__train

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32792,31,32,30,34,35,36,65567,38,39,40,...,65352,98120,98122,32587,98124,32589,32584,32591,65359,163663,32596,32598,32600,32602,98154,130922,98175,32649,65418,130956,98193,32657,32659,98203,130978,32686,65465,98239,98243,131013,131015,98248,32721,163809,98275,98279,65514,131050,65518,98296
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1724,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Построение модели на основе SDV**

In [27]:
%%time
U, S, VT = np.linalg.svd(user_item_matrix__train.T)
V = VT.T

CPU times: user 1min 35s, sys: 3.48 s, total: 1min 39s
Wall time: 51.6 s


In [28]:
# Матрица соотношения между пользователями и латентными факторами
U.shape

(10475, 10475)

In [29]:
# Матрица соотношения между объектами и латентными факторами
V.shape

(1725, 1725)

In [30]:
S.shape

(1725,)

In [31]:
Sigma = np.diag(S)
Sigma.shape

(1725, 1725)

In [32]:
# Диагональная матрица сингулярных значений
Sigma

array([[6.44242259e+02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.90612469e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.30453178e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.21000120e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.63516800e-14, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.12554711e-14]])

In [33]:
# Используем 3 первых сингулярных значения
r=3
Ur = U[:, :r]
Sr = Sigma[:r, :r]
Vr = V[:, :r]

In [34]:
# Матрица соотношения между новым пользователем и латентными факторами
test_user = np.mat(user_item_matrix__test.values)
test_user.shape, test_user

((1, 10475), matrix([[1.5, 2.5, 0. , ..., 0. , 0. , 0. ]]))

In [35]:
tmp = test_user * Ur * np.linalg.inv(Sr)
tmp

matrix([[-0.03210987, -0.00074386,  0.00198715]])

In [36]:
test_user_result = np.array([tmp[0,0], tmp[0,1], tmp[0,2]])
test_user_result

array([-0.03210987, -0.00074386,  0.00198715])

In [37]:
# Вычисляем косинусную близость между текущим пользователем и остальными пользователями
cos_sim = cosine_similarity(Vr, test_user_result.reshape(1, -1))
cos_sim[:10]

array([[0.51386506],
       [0.27446399],
       [0.58918284],
       [0.77620347],
       [0.43780319],
       [0.68410241],
       [0.39757477],
       [0.82922485],
       [0.84720479],
       [0.10977591]])

In [38]:
# Преобразуем размерность массива
cos_sim_list = cos_sim.reshape(-1, cos_sim.shape[0])[0]
cos_sim_list[:10]

array([0.51386506, 0.27446399, 0.58918284, 0.77620347, 0.43780319,
       0.68410241, 0.39757477, 0.82922485, 0.84720479, 0.10977591])

In [39]:
# Находим наиболее близкого пользователя
recommended_user_id = np.argsort(-cos_sim_list)[0]
recommended_user_id

855

In [40]:
movieId_list = list(user_item_matrix.columns)

df_name=df1[['MAL_ID','Name']]
df_merge = pd.merge(df_user, df_name, left_on='movieId', right_on='MAL_ID', how='inner')
df_merge.drop(columns=['MAL_ID','timestamp'])


,userId,movieId,rating,Name
0,1,110,1.0,Chuuka Ichiban!
1,11,110,3.5,Chuuka Ichiban!
2,22,110,5.0,Chuuka Ichiban!
3,24,110,5.0,Chuuka Ichiban!
4,29,110,3.0,Chuuka Ichiban!
...,...,...,...,...
105874,1684,6328,3.5,Bondage Game: Shinsou no Reijoutachi - Shinsou...
105875,1699,1780,1.0,Choukou Tenshi Escalayer
105876,1699,2744,1.0,Potemayo
105877,1699,2982,3.0,Yuuwaku


In [85]:
# Аниме, которые оценивал текущий пользователь:

i=1
for idx, item in enumerate(np.ndarray.flatten(np.array(test_user))):
    if item > 0:
        title = df_merge.at[idx, 'Name']
        id=df_merge.at[idx, 'userId']
        print('{} - {} - {}'.format(id, title, item))
        if i==50:
            break
        else:
            i+=1

1 - Chuuka Ichiban! - 1.5
11 - Chuuka Ichiban! - 2.5
33 - Chuuka Ichiban! - 3.0
68 - Chuuka Ichiban! - 3.5
82 - Chuuka Ichiban! - 2.0
142 - Chuuka Ichiban! - 3.0
157 - Chuuka Ichiban! - 2.0
217 - Chuuka Ichiban! - 3.0
224 - Chuuka Ichiban! - 3.0
308 - Chuuka Ichiban! - 2.0
457 - Chuuka Ichiban! - 2.5
638 - Chuuka Ichiban! - 2.0
816 - Chuuka Ichiban! - 4.0
848 - Chuuka Ichiban! - 3.0
881 - Chuuka Ichiban! - 0.5
1002 - Chuuka Ichiban! - 3.0
1012 - Chuuka Ichiban! - 2.5
1041 - Chuuka Ichiban! - 3.0
1055 - Chuuka Ichiban! - 3.0
1060 - Chuuka Ichiban! - 2.0
1113 - Chuuka Ichiban! - 3.0
1231 - Chuuka Ichiban! - 3.0
1409 - Chuuka Ichiban! - 3.0
1438 - Chuuka Ichiban! - 2.0
1445 - Chuuka Ichiban! - 2.0
1627 - Chuuka Ichiban! - 3.0
1658 - Chuuka Ichiban! - 3.0
37 - Gunparade Orchestra - 3.0
146 - Gunparade Orchestra - 3.0
340 - Gunparade Orchestra - 3.0
376 - Gunparade Orchestra - 3.0
482 - Gunparade Orchestra - 2.0
483 - Gunparade Orchestra - 3.0
502 - Gunparade Orchestra - 4.5
506 - Gunparade

In [90]:
# Фильмы, которые оценивал наиболее схожий пользователь:
i=1
recommended_user_item_matrix = user_item_matrix.loc[[recommended_user_id+1]]
for idx, item in enumerate(np.ndarray.flatten(np.array(recommended_user_item_matrix))):
    if item > 0:
        title = df_merge.at[idx, 'Name']
        id=df_merge.at[idx, 'userId']
        print('{} - {} - {}'.format(id, title, item))
        if i==30:
            break
        else:
            i+=1

1 - Chuuka Ichiban! - 2.5
11 - Chuuka Ichiban! - 3.5
82 - Chuuka Ichiban! - 2.0
157 - Chuuka Ichiban! - 4.0
224 - Chuuka Ichiban! - 4.0
752 - Chuuka Ichiban! - 2.0
918 - Chuuka Ichiban! - 2.5
937 - Chuuka Ichiban! - 5.0
955 - Chuuka Ichiban! - 3.0
1080 - Chuuka Ichiban! - 3.0
1145 - Chuuka Ichiban! - 5.0
1299 - Chuuka Ichiban! - 4.5
1316 - Chuuka Ichiban! - 3.5
1338 - Chuuka Ichiban! - 2.5
1408 - Chuuka Ichiban! - 3.5
1409 - Chuuka Ichiban! - 4.0
1438 - Chuuka Ichiban! - 5.0
1445 - Chuuka Ichiban! - 1.5
1465 - Chuuka Ichiban! - 1.0
1490 - Chuuka Ichiban! - 2.5
1718 - Chuuka Ichiban! - 1.0
778 - Kimi ga Nozomu Eien - 3.0
1055 - Kimi ga Nozomu Eien - 0.5
37 - Gunparade Orchestra - 4.0
146 - Gunparade Orchestra - 2.5
201 - Gunparade Orchestra - 4.0
249 - Gunparade Orchestra - 3.5
376 - Gunparade Orchestra - 5.0
480 - Gunparade Orchestra - 3.5
483 - Gunparade Orchestra - 4.0
